In [142]:
import pandas as pd
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification
from transformers import pipeline
from ktrain import text as txt


ts = txt.TransformerSummarizer()
ts2 = pipeline("summarization", model="t5-large", tokenizer="t5-large", framework="tf")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("model")

In [143]:
def summarize(text):
    return ts2(text, min_length=42, clean_up_tokenization_spaces=True, max_length=1024)[0]['summary_text']

In [2]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

In [3]:
businesses = pd.read_json("s3://yelp-dataset-pt-9/data/json/business.json", orient="records", lines=True)

In [4]:
reviews = pd.read_csv("s3://yelp-dataset-pt-9/data/csv/reviews.csv")

In [5]:
businesses.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [6]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0,5.0,0.0,0.0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0,1.0,1.0,0.0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0,1.0,0.0,0.0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,0.0,0.0,0.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0.0,0.0,0.0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [7]:
reviews.shape

(8021124, 9)

In [8]:
business_counts = reviews['business_id'].value_counts()

In [9]:
business_counts.sort_values(ascending=False).head()

RESDUcs7fIiihp38-d6_6g    10417
4JNXUYY8wbaaDmk3BPzlWw     9536
K7lWdNUhCbcnEvI0NhGewg     7594
f4x1YBxkLrZg652xt2KR5g     6859
cYwJA2A6I12KNkm2rtXd5g     5586
Name: business_id, dtype: int64

In [10]:
random_business = reviews[reviews['business_id'] == 'cYwJA2A6I12KNkm2rtXd5g']

In [11]:
random_business.shape

(5586, 9)

In [12]:
random_business.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
7206912,hJrtkwflyh9iD_EjZhSlng,jMpHxQnZl4I7m3HZVJVpgQ,cYwJA2A6I12KNkm2rtXd5g,4.0,0.0,0.0,0.0,Have been here twice now and both times it was...,2013-09-04 00:24:27
7207150,hclGbDgHjSz2WjgKdd6CGQ,q8BT_2-vBJNqm8NiW-hpeA,cYwJA2A6I12KNkm2rtXd5g,5.0,0.0,0.0,0.0,Absolutely wonderful food. Best burger with av...,2015-10-07 22:51:41
7207212,uwAyexoEmapukI0h5BTsJw,TeSXdvgDlVQkuOII1lAxmg,cYwJA2A6I12KNkm2rtXd5g,5.0,0.0,0.0,0.0,Not one for long reviews. This place rocks. On...,2014-01-17 02:36:21
7207336,IbLgz6CamDtuN1OC-_o-Hg,N6STM9kxnOtxJFYtMLVmLQ,cYwJA2A6I12KNkm2rtXd5g,3.0,0.0,0.0,0.0,Good for the price. \nFarm burger: bun very go...,2014-10-11 05:07:55
7207414,WDrbAVMYtrGbv--M-4drgA,Rk2OyIscXuYb_E19MxEj0Q,cYwJA2A6I12KNkm2rtXd5g,5.0,0.0,0.0,0.0,The food was delicious. The service was excel...,2016-07-25 06:05:55


In [13]:
random_business_avg_rating = random_business['stars'].mean()

In [14]:
random_business_med_rating = random_business['stars'].median()

In [15]:
random_business_avg_rating

3.8947368421052633

In [16]:
random_business_med_rating

4.0

In [17]:
random_business_info = businesses[businesses['business_id'] == 'cYwJA2A6I12KNkm2rtXd5g']

In [18]:
random_business_info

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
201975,cYwJA2A6I12KNkm2rtXd5g,Gordon Ramsay BurGR,3667 Las Vegas Blvd S,Las Vegas,NV,89109,36.110724,-115.172169,4.0,5494,0,"{'Caters': 'False', 'RestaurantsDelivery': 'Fa...","Burgers, Restaurants, American (Traditional)","{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ..."


In [19]:
from tqdm.notebook import tqdm
tqdm.pandas()

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [20]:
preds = random_business['text'].progress_apply(predict)

In [21]:
preds.mean()

3.8947585783485352

In [22]:
business_counts[business_counts < 100]

ogWjp5Ys4fQoDUOU9r7kAA    99
j7HO1YeMQGYo3KibMXZ5vg    99
BJRh7_SnU9zK9YzyO6FlHQ    99
voEuw3Nl0zbG_QYoFjBYFQ    99
F_oG1waKkS2CYDIZkOSmvA    99
                          ..
TTjBaAPhgIv6hCMesq3qZg     3
AZt6iqnR5dFG35hS9HKoyw     3
zxiQSd-AcciZqdInkxoVqw     3
8eVd_NK4Y6DuqiQtqcKRtg     3
YeHE15K6mLoojJZ2iUrBrA     3
Name: business_id, Length: 192679, dtype: int64

In [23]:
random_business = reviews[reviews['business_id'] == 'zWno83CCEC1cPGZvxQPf-g']

In [24]:
random_business_avg_rating = random_business['stars'].mean()

In [25]:
random_business_med_rating = random_business['stars'].median()

In [26]:
random_business_avg_rating, random_business_med_rating

(4.0, 5.0)

In [27]:
random_business_info = businesses[businesses['business_id'] == 'zWno83CCEC1cPGZvxQPf-g']

In [28]:
random_business_info

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
13723,zWno83CCEC1cPGZvxQPf-g,Costco Wholesale,3411 St Rose Pkwy,Henderson,NV,89052,35.98737,-115.138768,4.0,92,1,"{'DogsAllowed': 'False', 'ByAppointmentOnly': ...","Shopping, Tires, Gas Stations, Automotive, Who...","{'Monday': '10:0-20:30', 'Tuesday': '10:0-20:3..."


In [29]:
preds = random_business['text'].progress_apply(predict)

In [30]:
preds.mean()

3.9599697324964733

In [31]:
il = businesses[businesses['state'] == "IL"]

In [32]:
il.shape

(2034, 14)

In [33]:
il['city'].unique()

array(['Champaign', 'Urbana', 'Monticello', 'Tolono', 'Urbana, Illinois',
       'Rantoul', 'Savoy', 'Saint Joseph', 'Mahomet', 'Villa Grove',
       'Ogden', 'Philo', 'Homer', 'Sidney', 'Tuscola', 'Thomasboro',
       'Joliet', 'St Joseph', 'Penfield', 'Fisher', 'Mansfield',
       'Gifford', 'Broadlands', 'Chicago', 'St. Joseph', 'Schaumburg',
       'Bridgeview', 'Dewey', 'Fithian'], dtype=object)

In [34]:
my_towns = ['Champaign', 'Urbana', 'Monticello', 'Tolono', 'Urbana, Illinois',
            'Rantoul', 'Savoy', 'Saint Joseph', 'Mahomet', 'Villa Grove', 'Ogden', 
            'Philo', 'Homer', 'Sidney', 'Tuscola', 'Thomasboro','Fisher', 'Mansfield',
            'Gifford', 'Broadlands', 'Chicago', 'St. Joseph']

In [41]:
cu = il[il['city'].isin(my_towns)]

In [42]:
cu

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14..."
88,UiyjUa8CQmygujhe-ZYlsw,Movers 4 Less Inc,,Champaign,IL,61822,40.127030,-88.241699,3.0,15,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Home Services, Movers","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
93,efdPLo00sOD-1Wf77hg5jQ,TruGreen Lawn Care,108 Petry Ct,Champaign,IL,61822,40.137287,-88.238436,2.0,13,1,{'BusinessAcceptsCreditCards': 'True'},"Shopping, Nurseries & Gardening, Tree Services...","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."
265,-LfTBo0oa_uD454ScEW2XA,Merry Ann's Diner,1 E Main St,Champaign,IL,61820,40.118133,-88.242900,3.0,47,0,"{'RestaurantsDelivery': 'True', 'WiFi': 'u'fre...","Restaurants, Diners, Sandwiches, Breakfast & B...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
361,1QG4JRQhm1pZQSPu7Axcog,Tatman's Towing,810 E Perkins Rd,Urbana,IL,61802,40.127897,-88.197377,3.5,6,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Home Services, Keys & Locksmiths,...","{'Monday': '0:0-23:59', 'Tuesday': '0:0-23:59'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209106,yRKrMWQxQaMeR18s5ajcYw,Arby's,1913 W Springfield Ave,Champaign,IL,61821,40.112422,-88.279457,2.0,7,1,"{'RestaurantsTakeOut': 'True', 'WiFi': ''no'',...","Sandwiches, Fast Food, Restaurants","{'Monday': '10:0-23:0', 'Tuesday': '10:0-23:0'..."
209147,HjCdJCPnxNtVY6wKbaB0PA,Champaign County Clerk,1776 East Washington St,Urbana,IL,61802,40.106404,-88.186691,4.0,4,1,None,Public Services & Government,None
209249,n8SUVgkC_hOaZ7FiJd7sHw,April's Florist,512 E John St,Champaign,IL,61820,40.109259,-88.230699,4.5,10,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Event Planning & Services, Wedding Planning, S...","{'Monday': '9:0-17:30', 'Tuesday': '9:0-17:30'..."
209353,jA-sEoLY9tqzwX7-wv09pA,Studio 9 Hair Design,701 Devonshire Dr,Champaign,IL,61820,40.088439,-88.254522,3.5,4,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Hotels & Travel, Beauty & Spas, Health Retreat...","{'Tuesday': '9:0-20:0', 'Wednesday': '9:0-20:0..."


In [37]:
smoke_shack = 'kCIvm-8hNfD-qrCxiWLsSQ'

In [38]:
random_business = reviews[reviews['business_id'] == smoke_shack]

In [39]:
preds = random_business['text'].progress_apply(predict)

In [40]:
preds.mean()

2.1243374586105346

In [43]:
cu.sort_values("review_count", ascending=False)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
5340,9MnbQg7kfb_WgxoV0hXKSQ,Black Dog Smoke & Ale House,201 N Broadway Ave,Urbana,IL,61801,40.113818,-88.207689,4.5,831,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'T...","Barbeque, Restaurants, American (Traditional),...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
2788,-fiUXzkxRfbHY9TKWwuptw,DESTIHL Restaurant & Brew Works,301 N Neil St,Champaign,IL,61820,40.118337,-88.243777,4.0,712,1,"{'GoodForKids': 'True', 'OutdoorSeating': 'Tru...","Gluten-Free, Food, American (New), Restaurants...","{'Monday': '10:0-10:15', 'Tuesday': '11:0-22:0..."
186234,VIJ2KiDKhUVhhpNylEIfog,Maize Mexican Grill,60 E Green Street,Champaign,IL,61820,40.110409,-88.238955,4.5,563,1,"{'Ambience': '{'romantic': False, 'intimate': ...","Mexican, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
85639,-zEpEmDfFQL-ph0N3BDlXA,Sakanaya,403 E Green St,Champaign,IL,61820,40.110167,-88.232925,4.5,450,1,"{'RestaurantsAttire': 'u'casual'', 'BusinessPa...","Japanese, Sushi Bars, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
22309,u8C8pRvaHXg3PgDrsUHJHQ,Papa Del's Pizza,1201 S Neil St,Champaign,IL,61820,40.103242,-88.244164,3.5,440,1,"{'RestaurantsDelivery': 'True', 'Alcohol': ''f...","Food, Food Delivery Services, Sandwiches, Even...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-23:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107066,yxMhVaB65N5VGEWSvU9Qzw,Super Pantry,407 N Dunlap Ave,Savoy,IL,61874,40.058890,-88.251482,1.5,3,0,{'BusinessAcceptsCreditCards': 'True'},"Food, Automotive, Gas Stations, Convenience St...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
56749,adMqhdXZmlZxUBjyyJEyiA,Property Management People,1153 County Rd 1500 E,Champaign,IL,61822,40.098162,-88.263955,3.5,3,1,{'BusinessAcceptsCreditCards': 'True'},"Property Management, Real Estate, Home Services","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
57636,HmF0AvgKSELlDpyLR-gJNQ,Crossroads Grooming & Boarding,1805 E Olympian Rd,Urbana,IL,61802,40.155924,-88.183193,5.0,3,1,None,"Pet Sitting, Pet Services, Pets, Veterinarians...","{'Monday': '15:30-17:30', 'Tuesday': '15:30-17..."
106582,KcWZCGu0W16R60sh5gYSCw,We Keep It Kleen,901 Bluegrass Ln,Champaign,IL,61821,40.126256,-88.308345,3.5,3,1,None,"Home Services, Home Cleaning","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."


In [44]:
black_dog = '9MnbQg7kfb_WgxoV0hXKSQ'

In [45]:
random_business = reviews[reviews['business_id'] == black_dog]

In [46]:
random_business['stars'].mean(), random_business['stars'].median()

(4.414351851851852, 5.0)

In [47]:
preds = random_business['text'].progress_apply(predict)

In [51]:
preds.mean()

4.3564886809499175

In [48]:
pdels = 'u8C8pRvaHXg3PgDrsUHJHQ'

In [49]:
random_business = reviews[reviews['business_id'] == pdels]

In [50]:
random_business['stars'].mean(), random_business['stars'].median()

(3.3652561247216037, 4.0)

In [52]:
preds = random_business['text'].progress_apply(predict)

In [53]:
preds.mean()

3.3902328136503566

In [62]:
sorted(cu['name'].unique().tolist())[-100:]

['Urban Outfitters',
 'Urbana Acupuncture',
 'Urbana Boulders',
 'Urbana Butcher Shop',
 'Urbana Champaign Independent Media Center',
 'Urbana Dog Park',
 'Urbana Free Library',
 'Urbana Garden Family Restaurant',
 'Urbana Landmark Hotel',
 'Urbana Plumbing & Heating',
 'Urbana Sweetcorn Festival',
 'Urbana True Tires',
 'V Picasso',
 'V. Picasso',
 'VCA Heritage Animal Hospital',
 'Value Check',
 'Value Place Champaign',
 'Vape Vault',
 'Vapors',
 'Verizon',
 'Verizon Russell Cellular',
 'Verizon Wireless',
 'Vibez Performance Car Audio',
 'Village Inn Pizza',
 'Village Pet Doctor',
 'Village at Colbert Park',
 'Vineyard Church',
 "Vinny's By The Slice & More",
 'Vintage Karma',
 'Virginia Theatre',
 'Vision Fitness',
 'Voice KTV',
 'Volo Broadband',
 'W Eric Martin, DDS',
 'WackyNackies Gifts & Cafe',
 'Wal-Mart',
 'Walgreens',
 'Walmart',
 'Walmart Supercenter',
 'Walnut Street Tea',
 'Wampler Property Management',
 'Ward & Associates Realtors',
 "Watson's Shack & Rail",
 'Wcia-Chan

In [57]:
cu[cu['name'] == 'Champaign Ford City']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
119686,vdBpx5l5djH7yJ0SpOy27w,Champaign Ford City,701 W Marketview Dr,Champaign,IL,61822,40.136338,-88.254869,3.0,10,1,{'BusinessAcceptsCreditCards': 'False'},"Auto Repair, Used Car Dealers, Auto Parts & Su...","{'Monday': '9:0-19:0', 'Tuesday': '9:0-19:0', ..."


In [58]:
random_business = reviews[reviews['business_id'] == 'vdBpx5l5djH7yJ0SpOy27w']
random_business['stars'].mean(), random_business['stars'].median()

(3.0, 3.0)

In [59]:
random_business

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
4047431,76Corz-nWFsvHUIO8PXwzA,jC2F29-qV5gD2Ml5A2j48A,vdBpx5l5djH7yJ0SpOy27w,5.0,0.0,0.0,0.0,Great dealership with great staff! Washed and ...,2016-11-22 15:43:29
4142892,4i8HY_aUxiqnDwKrlYX5Mw,LhCabMt5hyHvVjlUJbYgnA,vdBpx5l5djH7yJ0SpOy27w,1.0,5.0,0.0,0.0,Purchased a 2012 Ford F150 Lariat. 6 months i...,2015-01-20 16:46:56
4158312,m1jQmFLU4BbAeAsdUIG7xQ,yZKeBqmlj0aswTdUNDRRrw,vdBpx5l5djH7yJ0SpOy27w,1.0,7.0,0.0,0.0,I have had the unfortunate pleaser of dealing ...,2015-03-07 22:05:10
4264214,DhE3gI0WytAvID93ze49PA,ZCMQue66H6HmCcqfpO1YRw,vdBpx5l5djH7yJ0SpOy27w,5.0,0.0,0.0,0.0,Thanks to the Service Team and Champaign Ford ...,2018-03-29 15:32:58
4316408,Ahat1iQNbDP4Mq1C5u2lpA,AWgNezxpjh6GTi0s40NNfw,vdBpx5l5djH7yJ0SpOy27w,5.0,0.0,0.0,0.0,I am leasing a 2016 Ford Escape. The leasing p...,2017-04-27 15:03:58
4324800,QWQ4439uDXg6A5nT3GOE6Q,OaaY-FJroA5tIuNVsb60Rw,vdBpx5l5djH7yJ0SpOy27w,5.0,0.0,0.0,0.0,The service staff are always very pleasant and...,2015-09-18 20:17:42
4393374,lE6irBR1hvWtWCJOr3TOyQ,Xw0ci1qpxkY1W30-LwJ8BQ,vdBpx5l5djH7yJ0SpOy27w,5.0,0.0,0.0,0.0,Absolute top notch dealership.! My salesman Ga...,2017-03-14 21:23:28
4477423,Lvhu9APBbOe4gN56WJhyJA,dUcgTY_FaSncNcYAxceZjA,vdBpx5l5djH7yJ0SpOy27w,1.0,2.0,0.0,0.0,I scheduled an appointment to get a recall and...,2016-01-25 22:52:37
4576869,L0evToYrZWRv_URb8XqeUQ,bpMuLZeK8yPEIc1ghpMMJw,vdBpx5l5djH7yJ0SpOy27w,1.0,4.0,0.0,0.0,Vehicle Customer Service was AND IS NOT there...,2016-02-11 01:00:34
4729753,PXpmx4hi3cWpp2U5nY9tuA,Dqx0fLbv3_DDVZON-KhSSw,vdBpx5l5djH7yJ0SpOy27w,1.0,0.0,0.0,0.0,I recently had to take my 2015 Ford Escape (26...,2018-05-30 20:58:15


In [60]:
preds = random_business['text'].progress_apply(predict)

In [61]:
preds.mean()

3.1195529460906983

In [64]:
cu[cu['name'] == "Watson's Shack & Rail"]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
83139,S_9OaE_RBWBIXnUKvxJKsQ,Watson's Shack & Rail,211 N Neil St,Champaign,IL,61820,40.117733,-88.243903,4.0,175,1,"{'Alcohol': 'u'full_bar'', 'DogsAllowed': 'Fal...","Breakfast & Brunch, American (Traditional), Ni...","{'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22..."


In [65]:
random_business = reviews[reviews['business_id'] == 'S_9OaE_RBWBIXnUKvxJKsQ']
random_business['stars'].mean(), random_business['stars'].median()

(4.005405405405406, 4.0)

In [68]:
random_business.sort_values("date", ascending=False)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
3123785,MxZ1-QP9u3O_CjNUb15tJw,rPRpQuZO_v6pfgXl_qQoMg,S_9OaE_RBWBIXnUKvxJKsQ,5.0,0.0,0.0,0.0,We came on a Saturday for our first visit. We ...,2019-12-09 18:29:27
3208819,EI_kNZeRwlwH_7wOqTdnGg,bcWi8kYnf_4CNzZTvHZ4cA,S_9OaE_RBWBIXnUKvxJKsQ,4.0,0.0,0.0,0.0,I've come to Watson shack on numerous occasion...,2019-11-21 19:35:51
3083595,Ja3oZ0XxwJeSNVyox5Twmw,_lo7B48MQ0s873QHs_GfZg,S_9OaE_RBWBIXnUKvxJKsQ,4.0,0.0,0.0,0.0,Beautiful day in Champaign and with our dog. T...,2019-10-26 01:54:06
3212354,rD_GQkbkwmmKUlBOMXLKFA,O0BOnYXyP0zGzVqDqIk5TA,S_9OaE_RBWBIXnUKvxJKsQ,5.0,0.0,0.0,0.0,The best place in Champaign-Urbana for chicken...,2019-10-06 12:11:44
3049440,Jgb94mIEMPPuS0XjyYVImQ,LTuuuAq-daa5Zh7NyrqdUg,S_9OaE_RBWBIXnUKvxJKsQ,3.0,0.0,0.0,0.0,I admit the service was excellent and my wife'...,2019-09-29 22:19:18
...,...,...,...,...,...,...,...,...,...
2951723,n1C9ByhoGOan_2Pv51pkdA,Rw27ITKImUlOZKQcEbA4nA,S_9OaE_RBWBIXnUKvxJKsQ,5.0,1.0,0.0,0.0,Mmm. Just what champaign needs. Our group sh...,2016-02-11 02:01:56
2952381,6m8j-8MK2WXRWeI9rZYPCQ,lqtxY4v8SPHLkkaJ9gc0lQ,S_9OaE_RBWBIXnUKvxJKsQ,5.0,1.0,0.0,0.0,Best fried chicken in the Midwest :) The gravy...,2016-02-10 19:48:38
3074405,hkQnA8u500UUfn2uq5ZA0w,Jl2UkzX4G12wsft3vKQ2AQ,S_9OaE_RBWBIXnUKvxJKsQ,2.0,4.0,3.0,2.0,I'll start my review by sending a shout out to...,2016-02-09 19:56:21
2756733,rq8UvR829QOnDqQ-Z3h-FQ,VqbEZAPmA4QuXOyD3-TItQ,S_9OaE_RBWBIXnUKvxJKsQ,5.0,1.0,0.0,0.0,They opened only three days ago. Our visit was...,2016-02-07 22:49:58


In [69]:
preds = random_business['text'].progress_apply(predict)

In [70]:
preds.mean()

4.00046689445908

In [71]:
cu.shape

(2027, 14)

In [72]:
cu_business_ids = cu['business_id'].unique()

In [89]:
cu_reviews = reviews[reviews['business_id'].isin(cu_business_ids)]

In [90]:
cu_reviews.shape

(47480, 9)

In [75]:
import csv
from sklearn.metrics import mean_absolute_error

In [78]:
def predict_and_print(df):
    true = []
    preds = []
    with open('cu_preds.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile)
        
        for idx, text, stars in tqdm(df[['text', 'stars']].itertuples(), total=len(df)):
            pred = float(predict(text))
            preds.append(pred)
            true.append(stars)
            spamwriter.writerow([idx, text, stars, pred])
            print(mean_absolute_error(true, preds), end="\r")
                
            
    return preds

In [79]:
preds = predict_and_print(cu_reviews)

0.35950051742937755


In [82]:
cu_preds = pd.read_csv("cu_preds.csv", header=None, names=['idx', 'text', 'stars', 'pred'])

In [83]:
cu_preds.head()

,idx,text,stars,pred
0,14,Called for a 5:15 morning reservation. The dri...,5.0,5.112949
1,108,"There are good things, and bad things about th...",2.0,2.203524
2,307,The building is lovely. The remodel after But...,1.0,1.553763
3,424,LOVE LOVE LOVE this place! I'm a bit of a suck...,5.0,4.849468
4,522,The bread company is perfect for lunch. The pr...,4.0,4.852670


In [91]:
cu_reviews['preds'] = preds

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [92]:
cu_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds
14,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5.0,1.0,0.0,0.0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,5.112949
108,n2BEHit2SwqsRo6jlTagUA,abdw-Nf5xxWdi5wmUO7UYg,2qLOnQtWSlTOtkfEFyWScg,2.0,2.0,0.0,0.0,"There are good things, and bad things about th...",2013-04-10 19:26:54,2.203524
307,m4AXzV9l14iFBd9DRdM82w,6X0i-oGUbh5DZdTHzFuKfg,9A1C1f0m4nQltQrOOTl-Kw,1.0,11.0,1.0,0.0,The building is lovely. The remodel after But...,2013-12-07 02:26:13,1.553763
424,A-yKlSLEQQcoHR5q2lCyHg,Yximlvn0cfb3yVDaLuXDxw,VHsNB3pdGVcRgs6C3jt6Zg,5.0,1.0,0.0,0.0,LOVE LOVE LOVE this place! I'm a bit of a suck...,2013-08-03 19:59:56,4.849468
522,asLnux3k2yqoVjcJcLuF0g,Cz8l7JQ2I_RhtK0BosH-7Q,Ah4i15g8Ow_zphzcpulTxQ,4.0,0.0,0.0,0.0,The bread company is perfect for lunch. The pr...,2015-12-05 05:38:55,4.852670


In [97]:
businesses = businesses.rename(columns={"stars": "bus_stars"})

In [99]:
cu_reviews_merged = cu_reviews.merge(businesses[['business_id', 'name', 'bus_stars', 'review_count']])

In [100]:
cu_reviews_merged.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count
0,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5.0,1.0,0.0,0.0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,5.112949,Black Cab & Limo,4.5,28
1,rPYl8Zlg9ywWRnU0FdEjcw,217u7Wad7y90x4el26-TIw,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,Called the night before for early morning pick...,2012-02-03 11:51:41,5.042127,Black Cab & Limo,4.5,28
2,Hgl0RjCqbEASn4glhIQYTA,XAmSOoytHosuAS2Y3IbZyA,ZlCSsWS07JulSBIQlrbF5w,5.0,5.0,0.0,0.0,Writing this review only a few minutes after I...,2013-08-16 10:49:02,4.685391,Black Cab & Limo,4.5,28
3,TrVEuR1rZ7Csx6dR2JcSGw,TbTtSLyUA8xBkHw-EEk07w,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,By far the best taxi service in Champaign. Fri...,2013-10-31 13:32:04,5.080356,Black Cab & Limo,4.5,28
4,ke0qiDkTmNuXaqHXFzpIrw,Vlm5IWkfq3YjlFElUj134A,ZlCSsWS07JulSBIQlrbF5w,5.0,0.0,1.0,0.0,I called for a pick up at 8pm the night before...,2014-02-26 16:04:38,5.085074,Black Cab & Limo,4.5,28


In [102]:
cu_reviews_merged.groupby("business_id")['stars'].agg(['mean', 'median'])

,mean,median
business_id,,
-05uZNVbb8DhFweTEOoDVg,2.000000,2.0
-1TUzO4XCmOeqi9eAGeDWA,4.333333,5.0
-2q4dnUw0gGJniGW2aPamQ,1.750000,1.5
-4ERSzsFQJ481DrMaubI-Q,2.333333,1.0
-5NXoZeGBdx3Bdk70tuyCw,3.894737,4.0
...,...,...
zro2Q5sOky54C_cJ176k3Q,2.750000,2.5
zrtRx2bm55zVeYIpzLLm3w,3.208333,4.0
zs4G7HIQvUVaZoWqwpVeEw,3.750000,4.5


In [108]:
cu_reviews_merged.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count
0,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5.0,1.0,0.0,0.0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,5.112949,Black Cab & Limo,4.5,28
1,rPYl8Zlg9ywWRnU0FdEjcw,217u7Wad7y90x4el26-TIw,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,Called the night before for early morning pick...,2012-02-03 11:51:41,5.042127,Black Cab & Limo,4.5,28
2,Hgl0RjCqbEASn4glhIQYTA,XAmSOoytHosuAS2Y3IbZyA,ZlCSsWS07JulSBIQlrbF5w,5.0,5.0,0.0,0.0,Writing this review only a few minutes after I...,2013-08-16 10:49:02,4.685391,Black Cab & Limo,4.5,28
3,TrVEuR1rZ7Csx6dR2JcSGw,TbTtSLyUA8xBkHw-EEk07w,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,By far the best taxi service in Champaign. Fri...,2013-10-31 13:32:04,5.080356,Black Cab & Limo,4.5,28
4,ke0qiDkTmNuXaqHXFzpIrw,Vlm5IWkfq3YjlFElUj134A,ZlCSsWS07JulSBIQlrbF5w,5.0,0.0,1.0,0.0,I called for a pick up at 8pm the night before...,2014-02-26 16:04:38,5.085074,Black Cab & Limo,4.5,28


In [113]:
cu_reviews_merged.groupby("business_id")['stars'].transform('mean')

0        4.5
1        4.5
2        4.5
3        4.5
4        4.5
        ... 
47475    3.0
47476    3.0
47477    3.0
47478    3.0
47479    3.0
Name: stars, Length: 47480, dtype: float64

In [114]:
cu_reviews_merged['mean_stars'] = cu_reviews_merged.groupby("business_id")['stars'].transform('mean')

In [115]:
cu_reviews_merged['median_stars'] = cu_reviews_merged.groupby("business_id")['stars'].transform('median')

In [116]:
cu_reviews_merged.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars
0,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5.0,1.0,0.0,0.0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,5.112949,Black Cab & Limo,4.5,28,4.5,5.0
1,rPYl8Zlg9ywWRnU0FdEjcw,217u7Wad7y90x4el26-TIw,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,Called the night before for early morning pick...,2012-02-03 11:51:41,5.042127,Black Cab & Limo,4.5,28,4.5,5.0
2,Hgl0RjCqbEASn4glhIQYTA,XAmSOoytHosuAS2Y3IbZyA,ZlCSsWS07JulSBIQlrbF5w,5.0,5.0,0.0,0.0,Writing this review only a few minutes after I...,2013-08-16 10:49:02,4.685391,Black Cab & Limo,4.5,28,4.5,5.0
3,TrVEuR1rZ7Csx6dR2JcSGw,TbTtSLyUA8xBkHw-EEk07w,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,By far the best taxi service in Champaign. Fri...,2013-10-31 13:32:04,5.080356,Black Cab & Limo,4.5,28,4.5,5.0
4,ke0qiDkTmNuXaqHXFzpIrw,Vlm5IWkfq3YjlFElUj134A,ZlCSsWS07JulSBIQlrbF5w,5.0,0.0,1.0,0.0,I called for a pick up at 8pm the night before...,2014-02-26 16:04:38,5.085074,Black Cab & Limo,4.5,28,4.5,5.0


In [117]:
cu_reviews_merged['mean_preds'] = cu_reviews_merged.groupby("business_id")['preds'].transform('mean')

In [118]:
cu_reviews_merged.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds
0,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5.0,1.0,0.0,0.0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,5.112949,Black Cab & Limo,4.5,28,4.5,5.0,4.566164
1,rPYl8Zlg9ywWRnU0FdEjcw,217u7Wad7y90x4el26-TIw,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,Called the night before for early morning pick...,2012-02-03 11:51:41,5.042127,Black Cab & Limo,4.5,28,4.5,5.0,4.566164
2,Hgl0RjCqbEASn4glhIQYTA,XAmSOoytHosuAS2Y3IbZyA,ZlCSsWS07JulSBIQlrbF5w,5.0,5.0,0.0,0.0,Writing this review only a few minutes after I...,2013-08-16 10:49:02,4.685391,Black Cab & Limo,4.5,28,4.5,5.0,4.566164
3,TrVEuR1rZ7Csx6dR2JcSGw,TbTtSLyUA8xBkHw-EEk07w,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,By far the best taxi service in Champaign. Fri...,2013-10-31 13:32:04,5.080356,Black Cab & Limo,4.5,28,4.5,5.0,4.566164
4,ke0qiDkTmNuXaqHXFzpIrw,Vlm5IWkfq3YjlFElUj134A,ZlCSsWS07JulSBIQlrbF5w,5.0,0.0,1.0,0.0,I called for a pick up at 8pm the night before...,2014-02-26 16:04:38,5.085074,Black Cab & Limo,4.5,28,4.5,5.0,4.566164


In [119]:
cu_reviews_merged['mae_pred_vs_true_mean'] = abs(cu_reviews_merged['mean_stars'] - cu_reviews_merged['mean_preds'])

In [120]:
cu_reviews_merged.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
0,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5.0,1.0,0.0,0.0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,5.112949,Black Cab & Limo,4.5,28,4.5,5.0,4.566164,0.066164
1,rPYl8Zlg9ywWRnU0FdEjcw,217u7Wad7y90x4el26-TIw,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,Called the night before for early morning pick...,2012-02-03 11:51:41,5.042127,Black Cab & Limo,4.5,28,4.5,5.0,4.566164,0.066164
2,Hgl0RjCqbEASn4glhIQYTA,XAmSOoytHosuAS2Y3IbZyA,ZlCSsWS07JulSBIQlrbF5w,5.0,5.0,0.0,0.0,Writing this review only a few minutes after I...,2013-08-16 10:49:02,4.685391,Black Cab & Limo,4.5,28,4.5,5.0,4.566164,0.066164
3,TrVEuR1rZ7Csx6dR2JcSGw,TbTtSLyUA8xBkHw-EEk07w,ZlCSsWS07JulSBIQlrbF5w,5.0,2.0,1.0,0.0,By far the best taxi service in Champaign. Fri...,2013-10-31 13:32:04,5.080356,Black Cab & Limo,4.5,28,4.5,5.0,4.566164,0.066164
4,ke0qiDkTmNuXaqHXFzpIrw,Vlm5IWkfq3YjlFElUj134A,ZlCSsWS07JulSBIQlrbF5w,5.0,0.0,1.0,0.0,I called for a pick up at 8pm the night before...,2014-02-26 16:04:38,5.085074,Black Cab & Limo,4.5,28,4.5,5.0,4.566164,0.066164


In [124]:
cu_reviews_merged['mae_pred_vs_true_mean'].value_counts().sort_index(ascending=False).head(50)

1.781347     3
1.337505     4
1.266613     3
1.192064     3
1.165520     3
1.100767     3
0.995771     3
0.978624     6
0.967515     4
0.954210     4
0.881139     3
0.862844     7
0.836127     3
0.806037     5
0.792103     3
0.782731     3
0.781238     3
0.770807     3
0.753984     5
0.735093     3
0.723341     3
0.709920     8
0.708548     4
0.708086     4
0.688628     3
0.686411     5
0.673351     4
0.652537     5
0.649523    10
0.645997     3
0.644974     4
0.637669     3
0.637065     3
0.633620     3
0.632242    12
0.631210     3
0.618034    16
0.614207     3
0.612662     5
0.612153     5
0.612041     3
0.605384     3
0.598055     3
0.597863     4
0.597672     3
0.592822     3
0.590388     3
0.584245     4
0.583575     4
0.578395     3
Name: mae_pred_vs_true_mean, dtype: int64

In [125]:
cu_reviews_merged['mae_pred_vs_true_mean'].mean()

0.10040186439227335

In [132]:
cu_reviews_merged.sort_values("mae_pred_vs_true_mean", ascending=False).head(50)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
42359,kBNio5e2qAZ41xCuZaJ1Lw,yZac3rePuoWGVTk0wZA5ww,n-7XCcAAXQkSZnq-ch0Hlg,1.0,2.0,1.0,0.0,I've gone to Dr. Glancy for over ten years for...,2016-05-14 18:19:32,5.013400,"Kevin Glancy, OD",2.5,3,2.333333,1.0,4.114680,1.781347
42358,Q5W-JLtwZkwJWwmS-5zuoA,PbTrwXkvyTKt7XkwSDSQSA,n-7XCcAAXQkSZnq-ch0Hlg,1.0,1.0,0.0,0.0,I have been receiving annual eye exams at Dr. ...,2016-05-30 18:06:10,2.772811,"Kevin Glancy, OD",2.5,3,2.333333,1.0,4.114680,1.781347
42357,ZctxZ62M_F8TRrBUlBVOSw,gjoMG9gYuNuhFFous17rSw,n-7XCcAAXQkSZnq-ch0Hlg,5.0,0.0,0.0,0.0,Dr. Glancy's office is great. I thought I had ...,2016-01-10 02:32:43,4.557831,"Kevin Glancy, OD",2.5,3,2.333333,1.0,4.114680,1.781347
18699,mfmY-xJqeMXOcNlA9C7btw,qntQp9UoeP6ju8D_W0Y0cw,4VYi3I-nVttZeOmrGEXHjQ,4.0,0.0,0.0,0.0,Store review:\nI am not sure why so many negat...,2014-07-11 15:12:08,4.587847,Wal-Mart,2.0,4,1.750000,1.0,3.087505,1.337505
18696,GTLhILwZ7Po8U-Gluwx9Nw,7lWo5nM84zG6QggxX4zouQ,4VYi3I-nVttZeOmrGEXHjQ,1.0,4.0,0.0,0.0,While there is nothing fantastically different...,2009-10-19 05:11:53,4.224284,Wal-Mart,2.0,4,1.750000,1.0,3.087505,1.337505
18697,86om3cLc6caW-ULOey6nCg,ZPm6gpIZtLdY1QvKHWIK4g,4VYi3I-nVttZeOmrGEXHjQ,1.0,3.0,3.0,1.0,Want a new set of tires for your car? 4 hours....,2007-06-15 17:09:08,1.750827,Wal-Mart,2.0,4,1.750000,1.0,3.087505,1.337505
18698,tqgKtbyvwjMfplqWVaQTDw,GxkY7BqazkQu6I9HBDpbqQ,4VYi3I-nVttZeOmrGEXHjQ,1.0,3.0,1.0,1.0,"This is the ""bad"" Wal-Mart in CU. It has the l...",2010-02-24 21:15:09,1.787061,Wal-Mart,2.0,4,1.750000,1.0,3.087505,1.337505
13186,H04RvDSMvBcSiRHgfvAHSQ,M7eLVk6qdNMC4ZPqcruCcw,JRVfV2ETohgAWKdSMr19MA,5.0,0.0,0.0,0.0,"Initially, I was hesitant to enroll my kids in...",2016-12-20 13:52:45,4.992563,The Little Gym,5.0,3,5.000000,5.0,3.733387,1.266613
13187,WfW0kmqLHHICtYd4Wsj2bA,kQcxxu1ReZ1z01Mp2IJxJw,JRVfV2ETohgAWKdSMr19MA,5.0,0.0,0.0,0.0,We love The Little Gym! Erin is so nice and a ...,2018-02-14 03:13:29,4.657408,The Little Gym,5.0,3,5.000000,5.0,3.733387,1.266613
13188,AOLYhproxj8CL8r8a_MlBQ,xo_6pdChEppXBEeqeUjqEA,JRVfV2ETohgAWKdSMr19MA,5.0,2.0,0.0,0.0,I miss little gym. I say that because they are...,2017-08-04 21:52:36,1.550191,The Little Gym,5.0,3,5.000000,5.0,3.733387,1.266613


In [187]:
bad_guess_bus = cu_reviews_merged[cu_reviews_merged['business_id'] == 'n-7XCcAAXQkSZnq-ch0Hlg']

In [188]:
bad_guess_bus.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
42357,ZctxZ62M_F8TRrBUlBVOSw,gjoMG9gYuNuhFFous17rSw,n-7XCcAAXQkSZnq-ch0Hlg,5.0,0.0,0.0,0.0,Dr. Glancy's office is great. I thought I had ...,2016-01-10 02:32:43,4.557831,"Kevin Glancy, OD",2.5,3,2.333333,1.0,4.11468,1.781347
42358,Q5W-JLtwZkwJWwmS-5zuoA,PbTrwXkvyTKt7XkwSDSQSA,n-7XCcAAXQkSZnq-ch0Hlg,1.0,1.0,0.0,0.0,I have been receiving annual eye exams at Dr. ...,2016-05-30 18:06:10,2.772811,"Kevin Glancy, OD",2.5,3,2.333333,1.0,4.11468,1.781347
42359,kBNio5e2qAZ41xCuZaJ1Lw,yZac3rePuoWGVTk0wZA5ww,n-7XCcAAXQkSZnq-ch0Hlg,1.0,2.0,1.0,0.0,I've gone to Dr. Glancy for over ten years for...,2016-05-14 18:19:32,5.013400,"Kevin Glancy, OD",2.5,3,2.333333,1.0,4.11468,1.781347


In [189]:
print(bad_guess_bus.iloc[1]['text'])

I have been receiving annual eye exams at Dr. Kevin Glancy & Associates for almost a decade now and have generally been pleased with both the quality of the optometrists and my interactions with the office staff. At my most recent eye exam in May 2016, Dr. Kim was terrific in explaining my options for glasses and contacts now that my eyes have crossed into bifocal territory. I really appreciated the time and care Dr. Kim took during my eye exam and the follow-up visit to make sure that the new contacts worked well. 

The problem is with the office staff. I have had multiple frustrating phone conversations where the office staff couldn't find my chart or any history of my appointment and kept me lingering on hold while they looked. Additionally, they ordered the wrong contacts for me and the office staff closed for the day *prior* to the posted closing time. In this latter case, my frustration was that I had just driven 30 miles to pick up contacts only to find their office closed when 

In [190]:
print(bad_guess_bus.iloc[2]['stars']), print(bad_guess_bus.iloc[2]['text'])

1.0
I've gone to Dr. Glancy for over ten years for my eye exams and contact lenses.  The prescription and fit have always been perfect and I've learned not to take this for granted.  One thing I really appreciate about Dr. Glancy is that he explains about how the eyes work as well as the contact lenses so that you understand the decisions he makes and why you need to take certain precautions to care for your eyes.  He's very professional and personable and his staff is as well!


(None, None)

In [133]:
bad_guess_bus = cu_reviews_merged[cu_reviews_merged['business_id'] == '4VYi3I-nVttZeOmrGEXHjQ']

In [134]:
bad_guess_bus.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
18696,GTLhILwZ7Po8U-Gluwx9Nw,7lWo5nM84zG6QggxX4zouQ,4VYi3I-nVttZeOmrGEXHjQ,1.0,4.0,0.0,0.0,While there is nothing fantastically different...,2009-10-19 05:11:53,4.224284,Wal-Mart,2.0,4,1.75,1.0,3.087505,1.337505
18697,86om3cLc6caW-ULOey6nCg,ZPm6gpIZtLdY1QvKHWIK4g,4VYi3I-nVttZeOmrGEXHjQ,1.0,3.0,3.0,1.0,Want a new set of tires for your car? 4 hours....,2007-06-15 17:09:08,1.750827,Wal-Mart,2.0,4,1.75,1.0,3.087505,1.337505
18698,tqgKtbyvwjMfplqWVaQTDw,GxkY7BqazkQu6I9HBDpbqQ,4VYi3I-nVttZeOmrGEXHjQ,1.0,3.0,1.0,1.0,"This is the ""bad"" Wal-Mart in CU. It has the l...",2010-02-24 21:15:09,1.787061,Wal-Mart,2.0,4,1.75,1.0,3.087505,1.337505
18699,mfmY-xJqeMXOcNlA9C7btw,qntQp9UoeP6ju8D_W0Y0cw,4VYi3I-nVttZeOmrGEXHjQ,4.0,0.0,0.0,0.0,Store review:\nI am not sure why so many negat...,2014-07-11 15:12:08,4.587847,Wal-Mart,2.0,4,1.75,1.0,3.087505,1.337505


In [135]:
print(bad_guess_bus.iloc[0]['text'])

While there is nothing fantastically different about this Walmart as compared to all the others, it successfully sucks the blood of our local businesses by importing half its goods from places where they pay 5-year-old girls 2 cents a day. Oh, but it's so cheap!


In [136]:
print(bad_guess_bus.iloc[1]['text'])

Want a new set of tires for your car? 4 hours.

Want to fill a prescription? 2 days.

Need help getting something out to your car? Better strap on that ol' backbrace.


In [145]:
predict("""While there is nothing fantastically different about this Walmart as compared to all the others, it successfully sucks the blood of our local businesses by importing half its goods from places where they pay 5-year-old girls 2 cents a day. Oh, but it's so cheap!""")

4.2242837

In [144]:
sumsummarizeize(bad_guess_bus.iloc[0]['text'])

'walmart imports half its goods from places where they pay 5-year-old girls 2 cents a day. the company also sucks the blood of our local businesses by importing.'

In [146]:
ts.summarize(bad_guess_bus.iloc[0]['text'])

"Walmart imports half of its goods from places where they pay 5-year-old girls 2 cents a day. Oh, but it's so cheap! Walmart is a local business that sucks the blood of our local businesses by importing half its goods. It's a local businesses that sucks our blood."

In [148]:
predict('walmart imports half its goods from places where they pay 5-year-old girls 2 cents a day. the company also sucks the blood of our local businesses by importing.')

1.2137918

In [150]:
print(bad_guess_bus.iloc[3]['text'])

Store review:
I am not sure why so many negative reviews, maybe I got lucky. I had ordered something for ship to store pick up (a tv stand). They called when it arrived, texted, and emailed. I came a few days later to pick it up. The lady was super friendly and after noticing I had sprained my wrist helped bring it out to the front without any hesitation. She helped lifted the box with the help of my dad into the car. Overall quick experience and friendly. I realize that people despise walmart for many things, but they do keep many people employed. Some things are a lot cheaper than other places. This place also use environmentally friendly sky lights to cut down on cost. 

Pros: cheaper prices, large selection

Cons: of course some staff maybe bad per other review, but I got lucky this time

Hidden deals:
1. Price match other store prices
2. Use coupons of course!
3. Free ship to store for pickup


In [151]:
bad_guess_bus = cu_reviews_merged[cu_reviews_merged['business_id'] == 'JRVfV2ETohgAWKdSMr19MA']
bad_guess_bus.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
13186,H04RvDSMvBcSiRHgfvAHSQ,M7eLVk6qdNMC4ZPqcruCcw,JRVfV2ETohgAWKdSMr19MA,5.0,0.0,0.0,0.0,"Initially, I was hesitant to enroll my kids in...",2016-12-20 13:52:45,4.992563,The Little Gym,5.0,3,5.0,5.0,3.733387,1.266613
13187,WfW0kmqLHHICtYd4Wsj2bA,kQcxxu1ReZ1z01Mp2IJxJw,JRVfV2ETohgAWKdSMr19MA,5.0,0.0,0.0,0.0,We love The Little Gym! Erin is so nice and a ...,2018-02-14 03:13:29,4.657408,The Little Gym,5.0,3,5.0,5.0,3.733387,1.266613
13188,AOLYhproxj8CL8r8a_MlBQ,xo_6pdChEppXBEeqeUjqEA,JRVfV2ETohgAWKdSMr19MA,5.0,2.0,0.0,0.0,I miss little gym. I say that because they are...,2017-08-04 21:52:36,1.550191,The Little Gym,5.0,3,5.0,5.0,3.733387,1.266613


In [152]:
print(bad_guess_bus.iloc[2]['text'])

I miss little gym. I say that because they are no longer with us. The previous owners left. We heard their were new owners but they never had a schedule. The next thing I knew they were selling equipment. My oldest got to enjoy little gym but his brother wont. I feel for the lost kids of little gym.


In [153]:
bad_guess_bus = cu_reviews_merged[cu_reviews_merged['business_id'] == 'Fo1VZNRK9Prl8oBlpSYXkA']
bad_guess_bus.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
5573,IbaTfRidZ0xCvPB160_-uw,QZRJE2h50B3V5YwqkPqhkA,Fo1VZNRK9Prl8oBlpSYXkA,4.0,4.0,0.0,0.0,Huge selection of generally cheap tools. They...,2011-04-29 06:12:56,4.171907,Harbor Freight Tools,2.5,3,2.666667,3.0,3.85873,1.192064
5574,QzPJy9hZxv5gHmc6NTLaew,NAJgBheNVQ6DudaLCBrJyw,Fo1VZNRK9Prl8oBlpSYXkA,3.0,1.0,0.0,0.0,Need tools for a job? Harbor Freight Tools car...,2016-08-09 15:50:20,4.167060,Harbor Freight Tools,2.5,3,2.666667,3.0,3.85873,1.192064
5575,vSLEeO37LdjBYx9zhrBuwA,iPdOCchBt_bTPMyl4c2YWA,Fo1VZNRK9Prl8oBlpSYXkA,1.0,0.0,2.0,0.0,I found the tool I needed in 1 minute. Howeve...,2015-01-31 18:13:08,3.237225,Harbor Freight Tools,2.5,3,2.666667,3.0,3.85873,1.192064


In [154]:
print(bad_guess_bus.iloc[1]['text'])

Need tools for a job? Harbor Freight Tools carries a large selection of affordable tools. Most everything is off brand and the quality might be lesser than a known brand but that doesn't mean it can't get the job done. For a fraction of the price (HFT $17 vs. Park $67), I found a set of T-handled Bondhus wrenches to work on my bike. They work beautifully. 

The staff is generally nice. People have helped me find specific items, were courteous when I had to exchange something (because I got the wrong one).

The store is located off a busy street and can be somewhat difficult to get to if you're not driving a car. There is plenty of car parking but no bike parking.


In [155]:
print(bad_guess_bus.iloc[2]['text'])

I found the tool I needed in 1 minute.  However, I stood in the checkout line for 15-20 min.


In [156]:
bad_guess_bus = cu_reviews_merged[cu_reviews_merged['business_id'] == 'tDTUSKxPUUkpFaxBZZl4Cw']
bad_guess_bus.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
37319,Ghnd27oyq9ndE-j5HGFxvQ,eEWXGa024EhqcXIXeNkTxw,tDTUSKxPUUkpFaxBZZl4Cw,5.0,2.0,0.0,0.0,"kay, i bought $74 worth of stuff here. the fr...",2010-05-18 05:40:19,3.387478,Aldi,5.0,6,5.0,5.0,4.137156,0.862844
37320,qCG-ssWZuRUUS-e55kt2pA,4Sg6t2dMolSl1OD6fNx7IA,tDTUSKxPUUkpFaxBZZl4Cw,5.0,0.0,0.0,1.0,"After comparing prices at several stores, I ha...",2009-06-22 18:58:05,4.371974,Aldi,5.0,6,5.0,5.0,4.137156,0.862844
37321,Iv1PdyukVJFx6cvHskz2sw,p-2tjwAQ_8pB5r552gQNFQ,tDTUSKxPUUkpFaxBZZl4Cw,5.0,2.0,0.0,0.0,The Aldi brands are just as tasty as the big b...,2011-07-23 14:30:06,4.131056,Aldi,5.0,6,5.0,5.0,4.137156,0.862844
37322,REymG975OouN5VZ3o7h7kQ,7of5R3aLB3OUyUPQFyHm5A,tDTUSKxPUUkpFaxBZZl4Cw,5.0,3.0,0.0,1.0,i've put off a review for Aldi for over a year...,2010-01-07 22:16:17,4.258974,Aldi,5.0,6,5.0,5.0,4.137156,0.862844
37323,GCaO66SBlaIEjTgou069Og,eEWXGa024EhqcXIXeNkTxw,tDTUSKxPUUkpFaxBZZl4Cw,5.0,1.0,0.0,0.0,"Okay, I just bought $74 worth of stuff at this...",2010-05-08 21:45:07,4.131872,Aldi,5.0,6,5.0,5.0,4.137156,0.862844


In [157]:
print(bad_guess_bus.iloc[0]['text'])

kay, i bought $74 worth of stuff here.  the frozen fish sticks are truly gross.  other than that, all my products were pretty tasty.  the asia egg rolls are kind of watery and tasteless, but edible.  a couple of "misses" on a very large purchase is A-OK when you are saving this much $$$.  MOST of my items were every bit as good as the regular grocery, and a few are actually tastier (frozen fries).


In [158]:
summarize(bad_guess_bus.iloc[0]['text'])

'a couple of "misses" on a very large purchase is a-OK when you are saving this much $$$. most of my items were every bit as good as the regular grocery. the frozen fish sticks are truly gross, but other than that, all my products were pretty tasty.'

In [164]:
predict('a couple of "misses" on a very large purchase is a-OK when you are saving this much $$$. most of my items were every bit as good as the regular grocery. the frozen fish sticks are truly gross, but other than that, all my products were pretty tasty.')

3.3281438

In [159]:
print(bad_guess_bus.iloc[1]['text'])

After comparing prices at several stores, I have found Aldi to be the cheapest the majority of the time.  The prices on meats and produce are often cheaper by a dollar or more and the location is excellent.


In [160]:
print(bad_guess_bus.iloc[2]['text'])

The Aldi brands are just as tasty as the big brands, and cheaper...way cheaper.  Good cereal, sauce, chocolate, hummus, bagels, bread, cheese, crackers (but stay away from the Rosemary and Olive Oil ones...ew), salsa, chips (of all varieties--including pita chips!), etc.  Frozen food is kind of "meh" but I guess so is most cheap frozen food (I'm talking frozen pizzas and burritos here).  The frozen vegetables are good, though.

I haven't bought many "fresh" foods here (fruits, vegetables), but the package of romaine lettuce I bought was fresh and crispy so I might start shopping for produce here too.

 And I just have to put a plug here for their hummus...it's my #1 snack food.  But I've realized that it's woefully expensive at regular grocery stores like Schnucks (as are many other types of food...apparently the northeast spoiled me?)  Thankfully Aldi has 3 varieties of hummus--garlic, tahini, and roasted red pepper--and they are all gloriously delicious.

Aside from getting a cart wi

In [161]:
print(bad_guess_bus.iloc[3]['text'])

i've put off a review for Aldi for over a year now... mostly because the first reviewer before me got it pretty much right. first off, Aldi has the best beer ($4 six-pack german lagers), riesling ($4 bottles, excellent stuff) and frozen tilapia (about $2 for 1/2 lb) at these prices in town. second, good mustard, salsa, milk, eggs, juice, dark chocolate, all for about $1 per jar, bottle, carton or whatever they come in. third, the fruit and cheese selection is not bad at all.

sure, this isn't a general-purpose grocery store. but it's hard not to find about 60% of everything you need for a grand total of $35 per trip, including the booze. i would not buy the chicken or beef at Aldi, since there are better quality selections around town. but if you avoid that (and maybe a few other things) it's hard not to love this place.

finally, this is the only store i know of in town that (a) charges a refundable quarter for every shopping cart, and (b) charges a few dimes for every plastic bag. ev

In [162]:
print(bad_guess_bus.iloc[4]['text'])

Okay, I just bought $74 worth of stuff at this Aldi.  I bought five or six paper bags and used them all, and some of the stuff wasn't even bagged.  I don't mind buying bags when I can save OVER HALF on my grocery bill.  And it's incredibly clean.  Putting a quarter in for a cart is well worth it.  Quite frankly, when you can buy this much stuff at an Aldi for this price, you know why it's thriving in a poor economy.


In [163]:
print(bad_guess_bus.iloc[5]['text'])

Love it!  Wanna save money and still get great food?  This is it.  Wish it was bigger.  Tortilla chips, pizza, german beer, best price on bananas anywhere, the chocolate is amazing, the cereal is as good as any major brand and 1/3 of the price.  Debit cards or cash only.


In [165]:
cu_reviews_merged.describe()

,stars,useful,funny,cool,preds,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
count,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000,47480.000000
mean,3.544061,1.312005,0.376685,0.314427,3.597420,3.560752,119.563290,3.544061,3.788753,3.597420,0.100402
std,1.505341,4.740490,1.729343,0.952492,1.432854,0.780706,168.350352,0.763224,1.125057,0.718050,0.097468
min,1.000000,0.000000,0.000000,0.000000,0.892121,1.000000,3.000000,1.000000,1.000000,0.993776,0.000319
25%,2.000000,0.000000,0.000000,0.000000,2.318104,3.000000,21.000000,3.104478,3.000000,3.222004,0.036049
50%,4.000000,0.000000,0.000000,0.000000,4.167592,3.500000,55.000000,3.650000,4.000000,3.717115,0.072866
75%,5.000000,2.000000,0.000000,0.000000,4.863362,4.000000,131.000000,4.092437,5.000000,4.105444,0.130821
max,5.000000,415.000000,103.000000,48.000000,5.162989,5.000000,831.000000,5.000000,5.000000,5.124519,1.781347


In [176]:
most_wrong = cu_reviews_merged[cu_reviews_merged['mae_pred_vs_true_mean'] > .5]

In [177]:
most_wrong.shape

(332, 17)

In [178]:
most_wrong.describe()

,stars,useful,funny,cool,preds,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
count,332.000000,332.000000,332.000000,332.000000,332.000000,332.000000,332.000000,332.000000,332.000000,332.000000,332.000000
mean,3.036145,1.912651,0.750000,0.665663,3.318448,3.022590,8.316265,3.036145,3.021084,3.318448,0.676222
std,1.569696,3.266819,2.186369,1.762216,1.420457,1.115099,9.435069,1.131911,1.438416,0.826007,0.202025
min,1.000000,0.000000,0.000000,0.000000,0.972075,1.000000,3.000000,1.000000,1.000000,1.552323,0.513577
25%,1.000000,0.000000,0.000000,0.000000,1.910130,2.000000,3.000000,2.195455,1.500000,2.734453,0.552323
50%,3.000000,1.000000,0.000000,0.000000,3.711485,3.000000,4.000000,3.028571,3.000000,3.539463,0.618034
75%,4.250000,2.000000,1.000000,1.000000,4.572329,4.000000,10.000000,4.000000,4.000000,3.923351,0.709920
max,5.000000,27.000000,22.000000,13.000000,5.153777,5.000000,34.000000,5.000000,5.000000,5.068426,1.781347


In [180]:
most_wrong_most_reviews = most_wrong[most_wrong['review_count'] > 10]

In [181]:
most_wrong_most_reviews.shape

(74, 17)

In [182]:
most_wrong_most_reviews['mae_pred_vs_true_mean'].mean()

0.5618515997319609

In [184]:
most_wrong_most_reviews.sort_values("review_count", ascending=False).head(50)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,preds,name,bus_stars,review_count,mean_stars,median_stars,mean_preds,mae_pred_vs_true_mean
39364,oXOp4H5tXEWT9tbFWsedpw,Ev55kUL77kZk6xCYam5www,QD61fmXTp476qfS0dpt2Lw,4.0,2.0,1.0,0.0,"I'm torn, because while the selection is very ...",2010-09-23 04:23:20,3.578548,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39338,TGi6cxY0tKSk3sYglSvoMA,LBX8NlkuvahBFAFCpp14pQ,QD61fmXTp476qfS0dpt2Lw,3.0,2.0,16.0,0.0,Let me sum up this place with one anecdote: I ...,2011-04-27 23:41:40,1.911045,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39344,5g3-0aTe3RI1-yqnE4xM_Q,Qy05ODNQSfSs3_ayfsSFyA,QD61fmXTp476qfS0dpt2Lw,4.0,5.0,1.0,9.0,I never felt compelled to write a review for S...,2013-06-13 18:57:57,4.911484,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39343,bOPHOgxCc2IikmlqTcqBBw,D7ypVwmT8535TsPslvVUQw,QD61fmXTp476qfS0dpt2Lw,2.0,8.0,0.0,0.0,first they got rid of the lunch counter. sigh...,2012-05-06 01:18:17,2.952993,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39342,J1x9aeZRjyo7JF_kHeeH-w,Myy-tV1aKksxkQ2R7QUkAQ,QD61fmXTp476qfS0dpt2Lw,4.0,1.0,0.0,0.0,What I love about Strawberry Fields is that it...,2007-08-02 02:45:35,4.577279,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39341,Q2BxZSrfFqByhtBYUAPv-g,N8Iriob3n95cfybxXF19DA,QD61fmXTp476qfS0dpt2Lw,3.0,4.0,2.0,2.0,I do most of my shopping at Common Ground or t...,2007-08-16 02:51:59,3.030884,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39340,DNiHNDIA1ZgIFWlJhtj3Zg,-YW5tN95MLiPRnwrEP2Z4w,QD61fmXTp476qfS0dpt2Lw,3.0,0.0,0.0,0.0,This place has some of the best bread in urban...,2014-12-03 16:16:27,4.313754,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39339,VtOBiINRffDVLvMaG3jtag,o3t_UG0ba7luzgy7oA8jVw,QD61fmXTp476qfS0dpt2Lw,4.0,1.0,0.0,0.0,I go to Strawberry Fields once a week for part...,2013-02-27 21:24:33,4.301388,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39337,lQGCqQg5s1K8XRLx1njBpg,LBX8NlkuvahBFAFCpp14pQ,QD61fmXTp476qfS0dpt2Lw,2.0,2.0,4.0,1.0,"ok, I still go in all the time, but the servic...",2012-04-04 01:14:56,1.991559,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577
39346,dKbAOUU_dd0B_XqyWfjTUA,SxBi6fjWUCyIeDZd8jSvmQ,QD61fmXTp476qfS0dpt2Lw,5.0,3.0,3.0,6.0,the original was a food co-op; started in the ...,2007-08-04 00:33:49,4.697518,Strawberry Fields,3.0,34,3.028571,3.0,3.542148,0.513577


In [185]:
review_text = """Best place in town to grab what you need in a hurry! The staff is great, friendly, and very informative! I've never been disappointed! Don't waste your time, go and see Shelby at the best dispensary in town!"""
reviews[reviews['text'] == review_text]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
3874497,2E7UyPLOsTQ8uu9_DFmk_A,EulkmLspt5mtkEG4mFMWkA,MpENa9cCNW3blkWQzmcHKg,1.0,0.0,0.0,0.0,Best place in town to grab what you need in a ...,2018-12-06 00:14:17


In [201]:
review_text = """Fireplace and breakfast anytime. Could not be happier. The place even had that wood burning campfire smell. Staff was friendly. I had a late breakfast and the eggs were perfect. The ham was fabulous."""
reviews[reviews['text'] == review_text]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
975995,JwIpdh5CHY08fx94L7CLyw,pbzeNgmK2qPhkZQ-M-7iCQ,qk_RGuKMuTcWMg5NZLas_w,4.0,0.0,0.0,0.0,Fireplace and breakfast anytime. Could not be ...,2015-02-07 06:20:11


In [202]:
businesses[businesses['business_id'] == 'qk_RGuKMuTcWMg5NZLas_w']

,business_id,name,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,attributes,categories,hours
38209,qk_RGuKMuTcWMg5NZLas_w,Cracker Barrel Old Country Store,6055 Som Center Rd,Willoughby,OH,44094,41.605039,-81.438829,3.0,35,1,"{'Caters': 'True', 'GoodForKids': 'True', 'Res...","Comfort Food, Event Planning & Services, Ameri...","{'Monday': '6:0-20:0', 'Tuesday': '6:0-22:0', ..."
